# Model 1 - MCMC with Turing.jl

The code below _should_ work when executed. I am having difficulties in getting the Turing.jl package to work with the Jupyter notebook, but it runs just fine for me when run in a Julia console session.

In [ ]:
using Turing
using Distributions, StatsFuns
using Mamba: describe, plot

# Create data ---------------------------------------------
data = Dict()
data["T"] = 100
data["k"] = [50, 51, 57, 55, 63, 62, 82, 94, 99, 100]
data["Δμ"] = [0.0100, 0.0215, 0.0464, 0.1000, 0.2154, 0.4642, 1.0000, 2.1544, 4.6416, 10.0000]


# Conduct inference ---------------------------------------
# First we are going to define a model. This is the way of doing it for the Turing package. 
# I managed to convert this over from the JAGS specification (see the original Matlab repo) 
# with not so much trouble. It's pretty nice in fact.
@model model1(k, Δμ, T) = begin
    Φ(x) = normcdf(0, 1, x)
    PC(Δμ, σ²) = Φ(Δμ/sqrt(2σ²))
    σ² ~ Uniform(0,100)
    for c = 1:length(k)
        k[c] ~ Binomial(T, PC(Δμ[c],σ²))
    end
    return σ²
end

# do sampling. Note that there are a variety of sampling algorithms we can specify
n_samples = 300
samples = sample( model1(data["k"], data["Δμ"], data["T"]), PG(50,n_samples))
σ²posterior = samples[:σ²]

# TODO: plot histogram of σ²posterior

# Posterior prediction ------------------------------------

function posterior_prediction(T::Int64, Δμ_list, σ²samples)
    Φ(x) = normcdf(0, 1, x)
    PC(Δμ, σ²) = Φ(Δμ/sqrt(2σ²))
    k_pred = zeros(n_samples, length(data["Δμ"]))
    k_pred = [rand(Binomial(T, PC(Δμ,σ²))) for Δμ in data["Δμ_list"], σ² in σ²samples]
end

k_pred = posterior_prediction(data["T"], data["Δμ"], σ²posterior)
# so now we have posterior predictive samples in k_pred.
# Each row corresponds to an mcmc sample, each column corresponds to a value of Δμ


# TODO: do summary stats or plotting of posterior predictions, and plot the actual data so we
# can do model critique visually

# TODO: can also do out of sample prediction simply by providing more values for Δμ

## References
Vincent, B. T. (2015). A tutorial on Bayesian models of perception. Journal of Mathematical Psychology, 66, 103–114. http://doi.org/10.1016/j.jmp.2015.02.001